In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 26 14:09:13 2020

E:各パラメタの配列(2次元)
F:現在の配列全体(3次元)
G:次の配列全体(3次元)

x:木の大きさ
theta:自分と周りの木の大きさの和の行列
"""

from __future__ import print_function
import os
import sys
import numpy as np
from scipy import signal
import cv2
import matplotlib.pyplot as plt
import random
import csv
import pprint

para_num = 3

In [2]:
def init_state_sample(width, height):
    v = np.zeros(((para_num, width, height)))
    v[1:3, :, :]=1.0
    
    return v

In [3]:
#入力：2次元各F　出力：2次元ラプラシアン
def Laplacian(u, mask):
    
    u_inflow_from_top = np.roll(a=u, shift=1, axis=0) * mask[0,1]
    u_inflow_from_bottom = np.roll(a=u, shift=-1, axis=0) * mask[2,1]
    u_inflow_from_left = np.roll(a=u, shift=1, axis=1) * mask[1,0]
    u_inflow_from_right = np.roll(a=u, shift=-1, axis=1) * mask[1,2]
    u_inflow_from_left_top = np.roll(a=u, shift=(1,1), axis=(0,1)) * mask[0,0]
    u_inflow_from_right_top = np.roll(a=u, shift=(1,-1), axis=(0,1)) * mask[0,2]
    u_inflow_from_left_bottom = np.roll(a=u, shift=(-1,1), axis=(0,1)) * mask[2,0]
    u_inflow_from_right_bottom = np.roll(a=u, shift=(-1,-1), axis=(0,1)) * mask[2,2]
    u_outflow = u * mask[1,1]
    lap = (u_inflow_from_top + u_inflow_from_bottom
           + u_inflow_from_left + u_inflow_from_right
           + u_inflow_from_left_top + u_inflow_from_right_top
           + u_inflow_from_left_bottom +  u_inflow_from_right_bottom
           + u_outflow)
    
    return lap

In [4]:
#引数：3次元配列
def next_generation_x(F,mask, Dx, killX):

    x_theta =  Laplacian(F[0], mask) / 0.01**2#N:自信と周囲1マスのxの和の行列
    
    
    partial_x = Dx * x_theta +  F[0] * F[0] * F[1] *  F[2] - F[0] * killX
        
    #値が上限超えた場合の処理
    #Fx = np.where(Fx < 0, 0, Fx) 
    #Fx = np.where(Fx > 1, 1, Fx)
    return partial_x

In [5]:
#引数：3次元配列
def next_generation_y(F,mask, Dy, feedY):
    #mask_num = np.count_nonzero(mask==1)

    y_theta =  Laplacian(F[1], mask) / 0.01**2#N:自信と周囲1マスのxの和の行列
    
    
    partial_y = Dy * y_theta - F[0] * F[0] * F[1] *  F[2] + feedY * (1.0-F[1])
              
    #値が上限超えた場合の処理
    #Fy = np.where(Fy < 0, 0, Fy) 
    #Fy = np.where(Fy > 1, 1, Fy)
    return partial_y

In [6]:
#引数：3次元配列
def next_generation_z(F, mask, Dz, feedZ):
    #mask_num = np.count_nonzero(mask==1)

    z_theta =  Laplacian(F[2], mask) / 0.01**2#N:自信と周囲1マスのxの和の行列
    
    
    partial_z = Dz * z_theta - F[0] * F[0] *  F[1] * F[2] + feedZ * (1.0 - F[2])
               
    #値が上限超えた場合の処理
    #Fx = np.where(Fz < 0, 0, Fz) 
    #Fx = np.where(Fz > 1, 1, Fz)
    return partial_z

In [7]:
def next_generation(F, mask, Dx ,Dy, Dz, killX, feedY, feedZ):
    partial_x = next_generation_x(F,  mask, Dx, killX)
    partial_y = next_generation_y(F,  mask, Dy, feedY)
    partial_z = next_generation_z(F,  mask, Dz, feedZ)
    F[0] += partial_x
    F[1] += partial_y
    F[2] += partial_z
    return F


In [8]:
#引数：2次元
def to_image(F, scale=5.0):
    #Fを離散化＝イメージごとに分類
    #tmp = classify_img(F)
    #print(tmp)
    #0.0～1.0を0～255に
    img = np.array(F*255, dtype=np.uint8)

    W = int(F.shape[1]*scale)
    H = int(F.shape[0]*scale)
    img = cv2.resize(img, (W, H), interpolation=cv2.INTER_NEAREST)
    return img

In [9]:
def show_graph(y):
    x = range(len(y))
    fig, ax = plt.subplots(1, 1)
    ax.set_ylim((-1000, 10000))
    line, = ax.plot(x, y, color='blue')
    plt.pause(0.01)
    # グラフをクリア
    line.remove()

In [10]:
def main():
    areaLength=50
    switch = 0
    ret = 0
    wait = 1
    t = 0
    period = 1
    mask_max = np.array([[0,1,0],[1,-4,1],[0,1,0]])
    para_flag = 1
    
    dx = 0.01
    Dx = 0.5e-5
    Dy = 2e-5
    Dz = 2e-5
    
    
    if para_flag == 3:
    #ネット
        killX = 0.042
        feedY = 0.056
        feedZ = 0.03
    elif para_flag ==2:
    #点に収束
        killX = 0.059
        feedY = 0.055
        feedZ = 0.030
    
    elif para_flag ==1:
    #伸びる線
        killX = 0.05
        feedY = 0.16
        feedZ = 0.068
    elif para_flag ==5:
    #花
        killX = 0.032
        feedY = 0.025
        feedZ = 0.020
    elif para_flag==0:
    #点・分裂
        killX = 0.036
        feedY = 0.013
        feedZ = 0.022
    elif para_flag ==4:
    #進む・ばらばら波
        killX = 0.015
        feedY = 0.005
        feedZ = 0.023
    elif para_flag ==6:
        killX = 0.04
        feedY = 0.01
        feedZ = 0.01
    
    SQUARE_SIZE = 6
    F = init_state_sample(areaLength, areaLength)
    
    square_start = areaLength // 2 - SQUARE_SIZE // 2
    square_end = areaLength // 2 + SQUARE_SIZE // 2
    F[0,square_start:square_end, square_start:square_end] = 0.5
    F[1,square_start:square_end, square_start:square_end] = 0.5
    F[2,square_start:square_end, square_start:square_end] = 0.5    
    
    while True:
        #描写
        img = to_image(F[0], scale=5.0)
        cv2.imshow("test", img)
      
        #period回実行
        i = 0
        while i < period:
            i = i + 1
            F = next_generation(F, mask_max, Dx ,Dy, Dz, killX+feedZ, feedY, feedZ)
                  
        ret = cv2.waitKey(wait)
        if ret == ord('r'):
            F = init_state_sample(areaLength, areaLength)
        if ret == ord('a'):
            F = init_state_random(areaLength, areaLength)
        if ret == ord('s'):
            wait = min(wait*2, 1000)
        if ret == ord('f'):
            wait = max(wait//2, 10)
        if ret == ord('q') or ret == 27:
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()